# Como saber se um cogumelo é comestível?
## Feito por: 
**André Costa e Guilherme Aranha**

## Introdução
É senso comum que cogumelos podem ser uma boa fonte de proteina para alimentação, mas sempre quando encotramos um, nunca sabemos se é possivel come-lo ou se ele pode apresentar riscos a sua saude tendo algum tipo de veneno. Assim, decidimos encontrar alguma base de dados com vairos cogumelos catalogados com suas caracteristicas fisicas e tambem se ele possui veneno ou nao.

In [45]:
#todas as bibliotecas que vamos utilizar nesse notebook
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, metrics, ensemble, naive_bayes, model_selection

*A base de dados que vamos utilizar foi encontrada no site Kraggle, sendo um arquivo csv com varios cogumelos catalogados
link:https://www.kaggle.com/uciml/mushroom-classification*

In [ ]:
#importand a planilha 
data = pd.read_csv("mushrooms.csv") 
data.head()

Como podemos observar, a planilha esta escrita em ingles, e os elementos delas estao apenas com as iniciais de seu significado.
Assim, para poder facilitar a manipulação dos dados, decidimos mudar ela para portugues e escrever por inteiro seus elementos

In [ ]:
data_clean = data

#alterando os titulos para portugues
colunas = {
    'class': 'tipo',
    'cap-shape': 'formato-do-chapeu',
    'cap-surface': 'superficie-do-chapeu',
    'cap-color': 'cor-do-chapeu',
    'bruises': 'marcas',
    'odor': 'cheiro',
    'gill-attachment': 'anexos-do-himenio',
    'gill-spacing': 'espacamento-do-himenio',
    'gill-size': 'tamanho-do-himenio',
    'gill-color': 'cor-do-himenio',
    'stalk-shape': 'formato-do-estipe',
    'stalk-root': 'raiz-do-estipe',
    'stalk-surface-above-ring': 'superficie-do-estipe-acima-do-anel',
    'stalk-surface-below-ring': 'superficie-do-estipe-abaixo-do-anel',
    'stalk-color-above-ring': 'cor-do-estipe-acima-do-anel',
    'stalk-color-below-ring': 'cor-do-estipe-abaixo-do-anel',
    'veil-type': 'tipo-do-veu',
    'veil-color': 'cor-do-veu',
    'ring-number': 'numero-de-aneis',
    'ring-type': 'tipo-do-anel',
    'spore-print-color': 'cor-da-esporada',
    'population': 'populacao',
    'habitat': 'habitat',
    }

data_clean = data_clean.rename(columns=colunas)

del data_clean['tipo-do-veu'] # <--- só tem um tipo, assim decidimos remover pois essa coluna nao agregaria nada no algoritmo

data_clean.head()

In [ ]:
#alterando os valores para portugues
valores = {
    'tipo': {'p': 'venenoso','e': 'comestivel',},
    'formato-do-chapeu': {'x': 'convexo','b': 'sino','s': 'afundado','f': 'plano','k': 'nodoso','c': 'conico',},
    'superficie-do-chapeu': {'s': 'lisa','y': 'escamoso','f': 'fibroso','g': 'ranhuras',},
    'cor-do-chapeu': {'n': 'marrom','y': 'amarelo','w': 'branco','g': 'cinza','e': 'vermelho','p': 'rosa','b': 'couro','u': 'roxo','c': 'canela','r': 'verde',},
    'marcas': {'t': 'sim','f': 'nao',},
    'cheiro': {'p': 'pungente','a': 'almondega','l': 'anis','n': 'nenhum','f': 'fedorento','c': 'queimado','y': 'peixe','s': 'apimentado','m': 'mofo',},
    'anexos-do-himenio': {'f': 'sem','a': 'com',},
    'espacamento-do-himenio': {'c': 'medio','w': 'perto',},
    'tamanho-do-himenio': {'n': 'pequeno','b': 'grande',},
    'cor-do-himenio': {'k': 'preto','n': 'marrom','g': 'cinza','p': 'rosa','w': 'branco','h': 'chocolate','u': 'roxo','e': 'vermelho','b': 'couro','r': 'verde','y': 'amarelo','o': 'laranja',},
    'formato-do-estipe': {'e': 'alargando','t': 'afunilando',},
    'raiz-do-estipe': {'e': 'igual','c': 'bastao','b': 'bulbo','r': 'enraizado','?': 'faltando',},
    'superficie-do-estipe-acima-do-anel': {'s': 'lisa','f': 'fibrosa','k': 'sedosa','y': 'escamosa',},
    'superficie-do-estipe-abaixo-do-anel': {'s': 'lisa','f': 'fibrosa','y': 'escamosa','k': 'sedosa',},
    'cor-do-estipe-acima-do-anel': {'w': 'branco','g': 'cinza','p': 'rosa','n': 'marrom','b': 'couro','e': 'vermelho','o': 'laranja','c': 'canela','y': 'amarelo',},
    'cor-do-estipe-abaixo-do-anel': {'w': 'branco','p': 'rosa','g': 'cinza','b': 'couro','n': 'marrom','e': 'vermelho','y': 'amarelo','o': 'laranja','c': 'canela',},
    'cor-do-veu': {'w': 'branco','n': 'marrom','o': 'laranja','y': 'amarelo',},
    'numero-de-aneis': {'o': '1','t': '2','n': '0',},
    'tipo-do-anel': {'p': 'pingente','e': 'evanescente','l': 'largo','f': 'resplandecente','n': 'nenhum',},
    'cor-da-esporada': {'k': 'preto','n': 'marrom','u': 'roxo','h': 'chocolate','w': 'branco','r': 'verde','o': 'laranja','y': 'amarelo','b': 'couro',},
    'populacao': {'s': 'espalhado','n': 'numeroso','a': 'abundante','v': 'varios','y': 'solitario','c': 'agrupados',},
    'habitat': {'u': 'urbano','g': 'grama','m': 'prado','d': 'floresta','p': 'caminhos','w': 'lixo','l': 'folhas',}
}

data_clean = data_clean.replace(valores)
data_clean.head()

Agora que a planilha já esta organizada e em portugues, vamos checar se existe algum cogumelo com as mesmas caracteristicas

In [ ]:
'Todos os cogumelos da lista: {} Cogumelos sem repetição: {}'.format(len(data_clean.index),len(data_clean.drop_duplicates().index))

Bom com isso sabemos que não existe nenhum cogumelo com as mesmas características na lista, portanto, todos eles sao únicos.
Assim, seria possivel fazer uma boa previsão se ele seria comestivel ou venenoso ja que não existe cogumelos com características iguais e resultados diferentes (comestivel ou venenoso).
Agora vamos observar as porcentagem de cada caracteristica no nosso dataset:

In [ ]:
i = 0 #posição do subplot
f = plt.figure(figsize=(20, 50)) #tamanho do subplot
for coluna in data_clean:
    i += 1
    plot = f.add_subplot(11, 2, i)
    plot.bar(data_clean[coluna].value_counts().index,data_clean[coluna].value_counts(normalize=True))
    plt.title(coluna)
plt.show()


Com isso reparamos que a distribuição entre cogumelos venenosos e comestiveis muito proxima, o que vai nos ajudar nas proximas etapas do trabalho pois esta bem equilibrado. Tambem reparamos que dentro de uma dominio, existem algumas caracteristicas que dominam como por exemplo a grande maioria dos cogumelos possuem anexos do himenio.
Vamos observar a distribuição entre cogumelos venenosos e comestiveis mais claramente:


In [ ]:
data_clean['tipo'].value_counts(normalize=True)

Assim, sabemos que nosso modelo previsivel deve ter uma taxa de acerto maior que 52%, ja que caso foi menor que isso, nosso modelo pode ser descartado.

Vamos invetigar agora a porcentagem de cogumelos venenosos e comestiveis por característica:

In [ ]:
for coluna in data_clean.drop(columns=['tipo']):
    pd.crosstab(index=data_clean[coluna], columns=data_clean['tipo'], normalize='columns').plot.bar()
    plt.title(coluna)
plt.show()

Concluimos a partir desses graficos que existem várias características diretamente relacionadas ao tipo de cogumelo. Chama expecial atenção o cheiro: se o cogumelo tiver algum cheiro, é possivel identificar se ele é venenoso a partir apenas dessa característica

Para criar nosso modelo, decidimos por fazer uma regressao logistica. Assim precisamos transformar todas nossas variaveis que sao qualitativas em quantitativas com ajuda da função get_dummies do pandas.

In [ ]:
data_clean_dummies = pd.get_dummies(data_clean)

In [ ]:
data_clean_dummies.head()

Como nossas variaveis ja estao no formato quantitavo para poder implementar nosso modelo, separamos nossos dados em 2 datasets com o intuito de usar um deles para treinar nosso modelo e outro para verificar a proeficiencia do nosso modelo.

In [ ]:
razao = 0.6 #razao é o tamanho do treino, 1-razao é o tamanho do teste
corte = int(razao*data_clean_dummies.shape[0])
data_treino = data_clean_dummies.iloc[:corte,:]
data_teste = data_clean_dummies.iloc[corte:,:]

In [ ]:
X_treino = data_treino.drop(columns=["tipo_comestivel", "tipo_venenoso"])
y_treino = data_treino['tipo_comestivel']
X_teste = data_teste.drop(columns=["tipo_comestivel", "tipo_venenoso"])
y_teste = data_teste['tipo_comestivel']

Desta forma, decidimos implementar 3 classificadores diferentes para analizar suas respostas, sendo eles o Random Forest, a regressao logistica e o Naive Bayes.

Optamos pelo Random Forest por ser um algoritmo relativamente simples de configurar, porem apresenta valores interessantes.
Esse nome surge pelo fato que o algorimo é um conjunto de arvores de decisao, com cada uma relativamente diferente do outro.
As arvores de decisao criam um fluxograma com condiçoes dos valores das variaveis indepentes, assim elas vao seguindo um ramo ate chegar em um resultado esperado (que no nosso caso seria comestivel = 1 e venenoso = 0). Como cada arvore é diferente uma da outra,o valor retornado da maioria das arvores seria o do nosso modelo. 

A regressao logistica seria a construção de uma função logistica que separaria os valores do grafico entre a variavel dependente e indepentente para prever o resultado do modelo (no nosso caso comestivel ou venenoso) pela posição do ponto no grafico. Assim ela é identica a uma regressao linear, mudando apenas o formato da função.

O Naive Bayes calcula caracterisitca por caracteristica para saber sua porcenntagem dela ser comestivel ou venenoso, assim apos fazer a multiplicação de todos esses valores para todas as caracterisitcas, ele nos informa qual classificação o nosso cogumelo tem mais chance de ser (no nosso caso venenoso ou comestivel). 

De todos esses modelos, a Random Forest possui alguns valores que alteram seu resultado. Para tornar o projeto simples, decidimos apenas modificar a quantidade de arvores geradas pelo modelo. 
Para isso vamos utilizar a validação cruzada, que testa o modelo com os diferentes numeros de arvores encontradas no dicionario dos parametros.

In [81]:
parametros = {
    'n_estimators': [10,30,50,70,100,120,150,200]
}

random_forest_cv = model_selection.GridSearchCV(ensemble.RandomForestClassifier(), param_grid=parametros, cv=50)
random_forest = random_forest_cv.fit(X_treino,y_treino)
random_forest.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=120,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Colocamos o cv do GridSearchCV alto no intuito de diminuir a variancia do resultado (ja que a random forest é um modelo randomico). E com isso percebemos que em geral, nosso modelo alcança os melhores resultados com apenas 120 arvores.

A partir disso, montamos os 3 algoritimos para analizar seus resultados a seguir

In [82]:
metodos = [ensemble.RandomForestClassifier(n_estimators=120), linear_model.LogisticRegression(), naive_bayes.GaussianNB()]

for i in metodos:

    model = i
    model.fit(X_treino, y_treino)

    ypred = model.predict(X_teste)

    resultado = pd.crosstab(ypred, y_teste, rownames=['predição:'], colnames=['é comestivel:'])
    acertos = resultado[0][0] + resultado[1][1]
    erros = resultado[0][1] + resultado[1][0]
    porcentagem = acertos/(acertos + erros)*100
    
    print('>>> ' + repr(i) + ':\n')
    print(f"acertos: {acertos} // erros: {erros} // %: {porcentagem}")
    display(resultado)

>>> RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=120,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False):

acertos: 3149 // erros: 101 // %: 96.89230769230768


é comestivel:,0,1
predição:,,
0,2459,93
1,8,690


>>> LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False):

acertos: 3131 // erros: 119 // %: 96.33846153846154


D:\comp\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


é comestivel:,0,1
predição:,,
0,2447,99
1,20,684


>>> GaussianNB(priors=None, var_smoothing=1e-09):

acertos: 2913 // erros: 337 // %: 89.63076923076923


é comestivel:,0,1
predição:,,
0,2416,286
1,51,497


Pelos resultados, todos os nossos modelos superaram seu benchmark (a porcentagem de cogumelos comestiveis na base de dados, 51.8%)

Todos os erros dos modelos tiveram peso maior nos falsos negativos, que no nosso caso é um motivo bom pois mais vale a classificação de um cogumelo como venenoso sendo ele comestivel do que ao contrario, pois isso poderia colocar em risco de morte.

Sendo de todos eles o melhor o Random Forest, porem seu grande problema é que por ser randomico, assim sua acuracia varia a cada teste, e nao conseguimos interpretar o modelo de decisao do algoritimo para entender seu funcionamento

Em Segundo Lugar foi a Regressao logistia, com aproximadamente 96% de acuracia. Apesar de nao ser o melhor, trouxe resultados bem proximos e sua vantagem é que podemos entender como o algoritmo esta classificando para possiveis melhoras caso precisamos ser mais preciso.

Em terceiro lugar foi o Naive Bayes, com aproximadamente 89% de acuracia. Isso era esperado pois para o Naive Bayes chegar em resultados interessantes, é preciso ter uma base de dados de treinamento bastante ampla, que nao era nosso caso. 

Assim, o problema de nossos modelos é que todos eles tentam classificar os cogumelos, mesmo que o resultado nao seja muito preciso. Isso no noso caso pode resultar em uma morte por um falso positivo. 
Alem disso, nossos modelos precisam de um computador ou matematica relativamente avançada para a classificação de um cogumelo, coisa que seria melhor nao depender caso alguem precise comer um cogumelo ou nao.
Levando tudo isso em conta, pretendemos apresentar nosso ultimo modelo que tem o intuito de fazer uma arvore de decisao que tenha muita precisao para saber se é possivel comer um cogumelo ou nao. (tudo isso utilizando a nosso base de dados, assim ainda estamos sujeito a teoria do cisne negro)


Para isso, vamos separar novamente a quantidade de cogumelos de cada caracteristica, para saber quantos sao venenosos e comestiveis

In [ ]:
for coluna in data_clean.drop(columns=['tipo']):
    display(pd.crosstab(index=data_clean[coluna], columns=data_clean['tipo']))


Com isso, conseguimos observar que existem apenas cogumelos com um tipo para certas caracteristicas, como por exemplo, so existe cogumelos comestiveis para cogumelos com o chapeu afundado.
Assim conseguimos saber que se um novo cogumelo tiver o chapeu afundado, sua maior probabilidade é que ele seja comestivel.
Desta forma, consegumos montar uma lista de todas as caracterisitcas que caso um cogumelo tenho, torna sua probabilidade de ser comestivel muito alta ou ate mesmo venenoso.

In [ ]:
confirma_venenoso = {}
confirma_comestivel = {}
for coluna in data_clean.drop(columns=['tipo']):
    for i in pd.crosstab(index=data_clean[coluna], columns=data_clean['tipo']).index:
        if pd.crosstab(index=data_clean[coluna], columns=data_clean['tipo']).loc[i,'comestivel'] == 0:
            confirma_venenoso.setdefault(coluna,[]).append(i)
        if pd.crosstab(index=data_clean[coluna], columns=data_clean['tipo']).loc[i,'venenoso'] == 0:
            confirma_comestivel.setdefault(coluna,[]).append(i)
print(confirma_venenoso)
print(confirma_comestivel)

A partir dessas duas listas, podemos montar um modelo facil de entender que qualquer um poderia usar caso encontre um cogumelo, ja que basta comparar as caracteristicas desse cogumelo com das listas para saber se ele é comestivel ou nao.
Agora vamos ao teste desse modelo para ver sua performance

In [ ]:
esperado = data_clean.iloc[:,0]
erros = []
conta = [0,0] 
for i in data_clean.index:
    result = ''
    for linha,coluna in zip(data_clean.drop(columns=['tipo']).iloc[i,:],data_clean.drop(columns=['tipo']).columns):
        if coluna in confirma_venenoso.keys():
            if linha in confirma_venenoso[coluna]:
                result = 'venenoso'
        if coluna in confirma_comestivel.keys():
            if linha in confirma_comestivel[coluna]:
                result = 'comestivel'
    if result == '':
        erros += [i]
    else:
        if result == esperado[i]:
            conta[0] += 1
        else:
            conta[1] += 1
comestiveis_perdidos = 0
for i in erros:
    if data_clean.iloc[i,0] == 'comestivel':
        comestiveis_perdidos += 1

print(f'acertos: {conta[0]} // erros : {conta[1]} // nao conseguiu: {len(erros)} [{"{:.2f}".format(100*len(erros)/data_clean.shape[0])}%] // cogumelos comestiveis perdidos : {comestiveis_perdidos} [{"{:.2f}".format(100*comestiveis_perdidos/len(erros))}%]')

Pelo resultado, percebemos que ele nao consegue classificar cerca de 18% dos cogumelos, porem como podemos observar, a grande maioria dos cogumelos que ele nao classifica sao comestiveis (98%), assim podemos ate assumir que caso um cogumelo nao entre em nenhuma das listas, é possivel assumir que ele é comestivel.

In [ ]:
esperado = data_clean.iloc[:,0]
erros = []
conta = [0,0] 
for i in data_clean.index:
    result = ''
    for linha,coluna in zip(data_clean.drop(columns=['tipo']).iloc[i,:],data_clean.drop(columns=['tipo']).columns):
        if coluna in confirma_venenoso.keys():
            if linha in confirma_venenoso[coluna]:
                result = 'venenoso'
    if result == '':
        result = 'comestivel'
    if result == esperado[i]:
        conta[0] += 1
    else:
        conta[1] += 1

print(f'acertos: {conta[0]} // erros : {conta[1]} [{"{:.2f}".format(100*conta[1]/data_clean.shape[0])}%]')

Assim, conseguimos ver que o erro é bem pequeno do nosso modelo (0,23%) e como, na pratica, nao temos certeza se essas caracteristicas de fato confirmam se um cogumelo é venenoso ou nao, podendo sempre estar sugeito a teoria do cisne negro. Concluimos que esse modelo seria o mais preciso que poderiamos chegar, sendo bem mais intuitivo que os outros utilizados anteriormente. Assim, para saber se é possivel consumir um cogumelo, é so comparar suas caracteristicas com a da lista a seguir, e caso ele tenha alguma caracteristica, a probabilidade dele ser venenoso é alta. 

In [83]:
confirma_venenoso

{'formato-do-chapeu': ['conico'],
 'superficie-do-chapeu': ['ranhuras'],
 'cheiro': ['apimentado',
  'fedorento',
  'mofo',
  'peixe',
  'pungente',
  'queimado'],
 'cor-do-himenio': ['couro', 'verde'],
 'cor-do-estipe-acima-do-anel': ['amarelo', 'canela', 'couro'],
 'cor-do-estipe-abaixo-do-anel': ['amarelo', 'canela', 'couro'],
 'cor-do-veu': ['amarelo'],
 'numero-de-aneis': ['0'],
 'tipo-do-anel': ['largo', 'nenhum'],
 'cor-da-esporada': ['verde']}